In [31]:
import psycopg2
import numpy as np

print('connecting to postgres...')
connection = psycopg2.connect(user="max",
                              password="password",
                              host="127.0.0.1",
                              port="5432",
                              database="final")
print()
print(connection)
cursor = connection.cursor()

connecting to postgres...

<connection object at 0x7eff6b4a7898; dsn: 'user=max password=xxx dbname=final host=127.0.0.1 port=5432', closed: 0>


In [32]:
q = """
select * from popular_spotify_features;
"""

cursor.execute(q)

In [33]:
spotify = cursor.fetchall()

In [34]:
len(spotify)

27787

so that's pretty messy. Let's load it up into a dataframe

In [35]:
import pandas as pd
import numpy as np

columns = [
    'lastfm_artist_track',
    'spotify_artist_track',
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode', 
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'uri',
    'analysis_url', 
    'duration_ms',
    'time_signature' 
]

df = pd.DataFrame(spotify, columns=columns)

# want to convert SQL Decimal(val) to float(val)
feats = [
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode', 
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo', 
    'duration_ms',
    'time_signature' 
]

df[feats] = df[feats].astype(float)
df = df.drop_duplicates('spotify_artist_track')
df.index = df['spotify_artist_track']
df.head()

,lastfm_artist_track,spotify_artist_track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,analysis_url,duration_ms,time_signature
spotify_artist_track,,,,,,,,,,,,,,,,,
!!! am/fm,!!! AM/FM,!!! am/fm,0.796,0.908,4.0,-6.188,0.0,0.0834,0.03870,0.3240,0.0450,0.676,115.981,spotify:track:0Y9SHiZGeXgShe4SKYl5sw,https://api.spotify.com/v1/audio-analysis/0Y9S...,295173.0,4.0
!!! all my heroes are weirdos,!!! All My Heroes Are Weirdos,!!! all my heroes are weirdos,0.505,0.992,4.0,-5.891,0.0,0.0918,0.00134,0.0537,0.0766,0.609,122.345,spotify:track:2nTla8UUqmBNKfcOvKc4bN,https://api.spotify.com/v1/audio-analysis/2nTl...,184080.0,4.0
!!! all u writers,!!! All U Writers,!!! all u writers,0.874,0.814,11.0,-6.873,0.0,0.0450,0.00412,0.4640,0.0690,0.741,121.976,spotify:track:2pPhbWZBm3Q6b35jV691qG,https://api.spotify.com/v1/audio-analysis/2pPh...,322623.0,4.0
!!! bend over beethoven,!!! Bend Over Beethoven,!!! bend over beethoven,0.630,0.927,9.0,-6.540,1.0,0.0457,0.00310,0.7390,0.1820,0.611,120.654,spotify:track:5PRM1a5qRJpsFzkz8VyRUY,https://api.spotify.com/v1/audio-analysis/5PRM...,486813.0,4.0
!!! break in case of anything,!!! Break in Case of Anything,!!! break in case of anything,0.708,0.913,4.0,-4.599,0.0,0.1040,0.02840,0.8170,0.3040,0.509,106.987,spotify:track:7owkuXT1aSBjCD3PoyHdDQ,https://api.spotify.com/v1/audio-analysis/7owk...,219840.0,4.0


In [36]:
df.shape

(26901, 17)

In [37]:
X = df[feats]
names = df.spotify_artist_track

In [38]:
df.shape

(26901, 17)

### baseline model: distance

In [39]:
# start by standard scalar
from sklearn.preprocessing import StandardScaler

std_scl = StandardScaler()
X_scaled = std_scl.fit_transform(X)

In [40]:
from data_collection import make_spotify_row
def find_n_nearest(song, n=15):
    try:
        song_feats = df[feats].loc[song].values
        song_feats = std_scl.transform(song_feats.reshape(1, -1))
    except KeyError:
        try:
            raw_feats = make_spotify_row(song)
        except:
            return f'{song} not in database'
        raw_feats = list(zip(raw_feats, columns))
        
        song_feats = []
        for t in raw_feats:
            value, feature = t
            if feature in feats:
                song_feats.append(value)
        song_feats = np.array(song_feats)
        
        song_feats = std_scl.transform(song_feats.reshape(1, -1))
    
    differences = X_scaled - song_feats
    distances = map(np.linalg.norm, differences)
    sorted_distances = sorted(enumerate(distances), key=lambda t: t[1])
    indices = [t[0] for t in sorted_distances]
    songs = df.iloc[indices]['spotify_artist_track'].values
    
    return songs[1:n+1]
    

In [41]:
find_n_nearest('kurt vile dust bunnies')

array(['bell x1 next to you', 'alphawezen speed of light',
       'angus & julia stone snow', 'aimee mann looking for nothing',
       'autoheart moscow', 'amen dunes time',
       'annie lennox walking on broken glass', 'backini go go killer',
       'alex cameron candy may',
       'au revoir simone another likely story (neon indian remix)',
       'big black delta money rain down',
       'a.a. bondy i can see the pines are dancing', 'beck jack-ass',
       'abra atoms', 'ac/dc hold me back'], dtype=object)

In [ ]:
print('blah')

In [42]:
df.iloc[-1]

lastfm_artist_track                 Camera Obscura Break It To You Gently
spotify_artist_track                camera obscura break it to you gently
danceability                                                        0.502
energy                                                              0.816
key                                                                     6
loudness                                                           -5.361
mode                                                                    1
speechiness                                                        0.0308
acousticness                                                      0.00925
instrumentalness                                                  9.6e-05
liveness                                                            0.332
valence                                                             0.601
tempo                                                             129.058
uri                                  s